In [3]:
import pandas as pd
import osmnx as ox
import networkx as nx
import math
from numpy import linalg as LA
import numpy as np

C:\Users\Willemijn Tutuarima\miniconda3\envs\gds\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


# File to calculate the weights of edges

In [1]:
filepath="../graph/rotterdam_drive_bbox_cameras_traffic_lights_bridges_roundabouts.graphml"

In [4]:
graph=ox.load_graphml(filepath)